### ECDSA Nonce Reuse - 200

In this challenge, you are given two ECDSA signatures that have reused the same nonce. Your task is to recover the private key used to generate these signatures. This is a classic cryptographic vulnerability that occurs when the same nonce is used in multiple ECDSA signatures.

```
r: 0x5d66e837a35ddc34be6fb126a3ec37153ff4767ff63cbfbbb32c04a795680491
```
```
s1: 0x1a53499a4aafb33d59ed9a4c5fcc92c5850dcb23d208de40a909357f6fa2c12c
```
```
message1: "what up defcon"
```
```
s2: 0xd67006bc8b7375e236e11154d576eed0fc8539c3bba566f696e9a5340bb92bee
```
```
message2: "uh oh this isn't good"
```
```
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
```


* s1 and s2 are the signature parameters.
* h1 and h2 are the message digests.
* r is the same nonce for both signatures due to nonce reuse.
* n is the order of the elliptic curve group.

Task: 
* Use the Python script for performing a ECDSA Nonce Reuse and recover the private key using the provided information and hints.

Once the private key is recovered, we as a attacker can use it to sign new transactions, effectively allowing them to transfer Bitcoin from the compromised wallet to our own addresses. This can lead to unauthorized spending of the victim's Bitcoin.

Hint:
* $ d_A = \left( \frac{s_2 \cdot h_1 - s_1 \cdot h_2}{r \cdot (s_1 - s_2)} \right) \mod n $. With this equation an attacker can obtain two different ECDSA signatures that reused the same nonce, they can use this equation to recover the private key $ d_A $. This allows the attacker to potentially forge signatures or decrypt messages that were intended to be secure.

In [11]:
from hashlib import sha256
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigdecode_string
from ecdsa.numbertheory import inverse_mod

def hash_message(message):
    """Hash the message using SHA-256."""
    message = 0

    assert len("{0:b}".format(message)) == 255, f"Expected binary length of 255, but got {len("{0:b}".format(message))}"
    return None

def recover_private_key(h1, h2, s1, s2, r1, r2, n):
    """Recover the private key via nonce reuse.

    Recover the private key from two different signatures
    that use the same random nonce `k` during signature
    generation. Note that if the same `k` is used in two
    signatures, this implies that the secp256k1 32-byte
    signature parameter `r` is identical.

    Parameters
    ----------
        h1: int
            The 32-byte message digest of the message `m1`.
        h2: int
            The 32-byte message digest of the message `m2`.
        s1: int
            The secp256k1 32-byte signature parameter `s1`.
        s2: int
            The secp256k1 32-byte signature parameter `s2`.
        r1: int
            The secp256k1 32-byte signature parameter `r1`.
        r2: int
            The secp256k1 32-byte signature parameter `r2`.
        n:  int
            The 32-byte integer order of G (part of the public key).

    Returns
    -------
        pk: int
            Return a 32-byte private key.

    """
    assert r1 == r2, "No ECDSA nonce reuse detected."
    return None

if __name__ == "__main__":
    """An illustrative recovery of the private key."""
    # Provided values
    r = 0x5d66e837a35ddc34be6fb126a3ec37153ff4767ff63cbfbbb32c04a795680491
    signature1 = 0x1a53499a4aafb33d59ed9a4c5fcc92c5850dcb23d208de40a909357f6fa2c12c
    signature2 = 0xd67006bc8b7375e236e11154d576eed0fc8539c3bba566f696e9a5340bb92bee
    n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

    # Messages
    message1 = "what up defcon"
    message2 = "uh oh this isn't good"

    # Step 1: Hash the messages
    z1 = hash_message(message1)
    z2 = hash_message(message2)

    # Step 2: Solve for the private key
    recovered_private_key = recover_private_key(z1, z2, signature1, signature2, r, r, n)

    print(f"Recovered private key: {recovered_private_key}")


AssertionError: Expected binary length of 255, but got 1

### Solution

#### ECDSA Nonce Reuse — 200
https://bitcoin.stackexchange.com/questions/35848/recovering-private-key-when-someone-uses-the-same-k-twice-in-ecdsa-signatures
* A classic ECDSA bad implementation attack : nonce reuse. 
* Here is my code used to solve it. Don’t forget to hash the messages and that there are 4 possibilities to try.

In [12]:
# Source: https://github.com/pcaversaccio/ecdsa-nonce-reuse-attack/tree/main

from hashlib import sha256
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigdecode_string
from ecdsa.numbertheory import inverse_mod

def hash_message(message):
    """Hash the message using SHA-256, 
       and convert to int using big-endian byte order"""
    return int.from_bytes(sha256(message.encode()).digest(), 'big')

def recover_private_key(h1, h2, s1, s2, r1, r2, n):
    """Recover the private key via nonce reuse.

    Recover the private key from two different signatures
    that use the same random nonce `k` during signature
    generation. Note that if the same `k` is used in two
    signatures, this implies that the secp256k1 32-byte
    signature parameter `r` is identical.

    Parameters
    ----------
        h1: int
            The 32-byte message digest of the message `m1`.
        h2: int
            The 32-byte message digest of the message `m2`.
        s1: int
            The secp256k1 32-byte signature parameter `s1`.
        s2: int
            The secp256k1 32-byte signature parameter `s2`.
        r1: int
            The secp256k1 32-byte signature parameter `r1`.
        r2: int
            The secp256k1 32-byte signature parameter `r2`.
        n:  int
            The 32-byte integer order of G (part of the public key).

    Returns
    -------
        pk: int
            Return the 32-byte private key.

    """
    assert r1 == r2, "No ECDSA nonce reuse detected."
    return ((s2 * h1 - s1 * h2) * inverse_mod((r1 * (s1 - s2)), n)) % n

if __name__ == "__main__":
    """An illustrative recovery of the private key."""
    # Provided values
    r = 0x5d66e837a35ddc34be6fb126a3ec37153ff4767ff63cbfbbb32c04a795680491
    signature1 = 0x1a53499a4aafb33d59ed9a4c5fcc92c5850dcb23d208de40a909357f6fa2c12c
    signature2 = 0xd67006bc8b7375e236e11154d576eed0fc8539c3bba566f696e9a5340bb92bee
    n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

    # Messages
    message1 = "what up defcon"
    message2 = "uh oh this isn't good"

    # Step 1: Hash the messages
    z1 = hash_message(message1)
    z2 = hash_message(message2)

    print(z1)
    print(z2)

    # Step 2: Solve for the private key
    recovered_private_key = recover_private_key(z1, z2, signature1, signature2, r, r, n)

    print(f"Recovered private key: {recovered_private_key}")


43900070582032049661984845976499951336967038182580098384543716376237677641549
38380581964160396214562900642339251000005602778589721372543316391429701613081
Recovered private key: 1337013370133701337


In [1]:
from Crypto.Hash import SHA256
# 1. Calculate the message hashes h1 and h2:
message1 = b"what up defcon" 
message2 = b"uh oh this isn't good"

h1 = int(SHA256.new(message1).hexdigest(), 16)
h2 = int(SHA256.new(message2).hexdigest(), 16)



# 2. Recover the private key using the formula:

r = 0x5d66e837a35ddc34be6fb126a3ec37153ff4767ff63cbfbbb32c04a795680491
s1 = 0x1a53499a4aafb33d59ed9a4c5fcc92c5850dcb23d208de40a909357f6fa2c12c  
s2 = 0xd67006bc8b7375e236e11154d576eed0fc8539c3bba566f696e9a5340bb92bee
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

d = ((h1 * s2 - h2 * s1) * pow(r * (s1 - s2), -1, n)) % n
print(d)


1337013370133701337
